# Child Mind Institute — Problematic Internet Use
**Relating Physical Activity to Problematic Internet Use**

The goal of this competition is to develop a predictive model that analyzes children's physical activity and fitness data to identify early signs of problematic internet use. Identifying these patterns can help trigger interventions to encourage healthier digital habits.

**Credits**

This solution was based on some sophisticated DNN that was the winning solution of the <a href="https://www.kaggle.com/competitions/icr-identify-age-related-conditions">ICR competition</a> 

<a href="https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/430843">Here</a> the solution explained by its author @ROOM722

His solution, in turn, was inspired by the following Keras code example:
<a href="https://keras.io/examples/structured_data/classification_with_grn_and_vsn/">Classification with Gated Residual and Variable Selection Networks</a>



In [ ]:
import pandas as pd
import numpy as np
import math
import os
import random
import gc
import pathlib
import matplotlib.pyplot as plt

from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import BayesianRidge, Ridge
from scipy import stats as st

import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras import regularizers as R
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import layers as L
from tensorflow.keras import optimizers as O
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
SAVE_MY_TRAIN = False
USE_MY_TRAIN = False
FULL_TRAIN = True
VALID = False
TEST = True  

In [ ]:
def set_randoms(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  

In [ ]:
SEED = 42
set_randoms(SEED)

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
%%time

# ideas taken from the following notebooks:
# https://www.kaggle.com/code/abdmental01/cmi-best-single-model
# https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-actigraphy-data-eda

def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df = df[df['non-wear_flag'] == 0]
    
    df['time_of_day_hours'] = (df['time_of_day'] / 1e9 / 3600)  # nanoseconds to hours
        
    stats = np.full((4, 6), fill_value=-1, dtype='float32') 
    stats_business_day = np.full((4, 3), fill_value=-1, dtype='float32') 
    stats_weekend = np.full((4, 3), fill_value=-1, dtype='float32') 
    for i in range(4):
        stats_business_day[i] = df[(df['weekday'].isin([1, 2, 3, 4, 5])) & (df['time_of_day_hours'] >= i * 6) & (df['time_of_day_hours'] < (i + 1) * 6)][['enmo']].agg(['mean', 'max', 'std']).values.reshape(-1)
        stats_weekend[i] = df[(df['weekday'].isin([6, 7])) & (df['time_of_day_hours'] >= i * 6) & (df['time_of_day_hours'] < (i + 1) * 6)][['enmo']].agg(['mean','max', 'std']).values.reshape(-1)
        stats[i] = np.concatenate([stats_business_day[i], stats_weekend[i]])

    return stats.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df

In [ ]:
def preprocess_df(df):
    df.loc[df['Physical-BMI'] == 0, 'Physical-BMI'] = None
    df.loc[df['Physical-Weight'] == 0, 'Physical-Weight'] = None
    df.loc[df['Physical-Diastolic_BP'] == 0, 'Physical-Diastolic_BP'] = None
    df.loc[df['Physical-Systolic_BP'] == 0, 'Physical-Systolic_BP'] = None
    
    seasons = {'Winter': 0, 'Fall': 1, 'Spring': 2, 'Summer': 3}
    season_cols = [c for c in df.columns if 'Season' in c]
    for c in season_cols:
        df[c] = df[c].map(seasons)
    
    return df

In [ ]:
train_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv', index_col='id')
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
train_df = pd.merge(train_df, train_ts, how="left", on='id').set_index('id')
train_df = preprocess_df(train_df)

# some data corrections
train_df.drop(['83525bbe'], axis=0, inplace=True)
train_df.loc['464a75fb', 'Physical-Diastolic_BP'] = 79.0
train_df.loc[['967d790c', '77a5b2ad'], 'Physical-Systolic_BP'] = None
train_df.loc['d819548d', 'Physical-Systolic_BP'] = 173.0
train_df.loc['f2499682', 'Fitness_Endurance-Time_Mins'] = None

train_df.drop(columns=[c for c in train_df.columns if c.startswith('PCIAT-')], inplace=True)

test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")
test_df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv', index_col='id')
test_df = pd.merge(test_df, test_ts, how="left", on='id').set_index('id')
test_df = preprocess_df(test_df)

season_cols = [c for c in train_df.columns if 'Season' in c]
features_to_transform = [f for f in train_df.columns if f not in season_cols and f != 'sii']
scaler = MinMaxScaler()
scaler.fit(pd.concat([train_df[features_to_transform], test_df[features_to_transform]], ignore_index=True))
# scaler.fit(train_df[features_to_transform])
train_df[features_to_transform] = scaler.transform(train_df[features_to_transform])
test_df[features_to_transform] = scaler.transform(test_df[features_to_transform])

features_to_impute = [f for f in train_df.columns if f != 'sii']
imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=25, 
                           random_state=0)   
imputer.fit(pd.concat([train_df[features_to_impute], test_df[features_to_impute]], ignore_index=True))
# imputer.fit(train_df[features_to_impute])
train_df_filled = pd.DataFrame(imputer.transform(train_df[features_to_impute]), columns=features_to_impute, 
                                                         index=train_df.index)
train_df_filled['sii'] = train_df['sii']
train_df = train_df_filled.dropna(subset=['sii']).reset_index()

if TEST:
    test_df_filled = pd.DataFrame(imputer.transform(test_df), columns=test_df.columns, index=test_df.index)
    test_df = test_df_filled

In [ ]:
train_df.describe()

In [ ]:
@tf.keras.utils.register_keras_serializable()
def smish(x):
    return x * K.tanh(K.log(1 + K.sigmoid(x)))


@tf.keras.utils.register_keras_serializable()
class GatedLinearUnit(L.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.linear = L.Dense(units)
        self.sigmoid = L.Dense(units, activation="sigmoid")
        self.units = units

    def get_config(self):
        config = super().get_config()
        config['units'] = self.units
        return config
    
    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)
    

@tf.keras.utils.register_keras_serializable()
class GatedResidualNetwork(L.Layer):
    def __init__(self, units, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.dropout_rate = dropout_rate
        self.relu_dense = L.Dense(units, activation=smish)
        self.linear_dense = L.Dense(units)
        self.dropout = L.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = L.LayerNormalization()
        self.project = L.Dense(units)

    def get_config(self):
        config = super().get_config()
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        return config
    
    def call(self, inputs):
        x = self.relu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x
    

@tf.keras.utils.register_keras_serializable()
class VariableSelection(L.Layer):
    def __init__(self, num_features, units, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = L.Dense(units=num_features, activation="softmax")
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate

    def get_config(self):
        config = super().get_config()
        config['num_features'] = self.num_features
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        return config
    
    def call(self, inputs):
        v = L.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input_ in enumerate(inputs):
            x.append(self.grns[idx](input_))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs
    

@tf.keras.utils.register_keras_serializable()
class VariableSelectionFlow(L.Layer):
    def __init__(self, num_features, units, dropout_rate, dense_units=None, **kwargs):
        super().__init__(**kwargs)
        self.variableselection = VariableSelection(num_features, units, dropout_rate)
        self.split = L.Lambda(lambda t: tf.split(t, num_features, axis=-1))
        self.dense = dense_units
        if dense_units:
            self.dense_list = [L.Dense(dense_units, \
                                       activation='linear') \
                               for _ in tf.range(num_features)
                              ]
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate
        self.dense_units = dense_units
        
    def get_config(self):
        config = super().get_config()
        config['num_features'] = self.num_features
        config['units'] = self.units
        config['dropout_rate'] = self.dropout_rate
        config['dense_units'] = self.dense_units
        return config        
    
    def call(self, inputs):
        split_input = self.split(inputs)
        if self.dense:
            l = [self.dense_list[i](split_input[i]) for i in range(len(self.dense_list))]
        else:
            l = split_input
        return self.variableselection(l)    

In [ ]:
n_splits = 10
batch_size = 32

units_1 = 32
drop_1 = 0.75
dense_units = 0 # 8

units_2 = 16
drop_2 = 0.5

units_3 = 8
drop_3 = 0.25

In [ ]:
learning_rate = 0.001

num_epochs = 10 if FULL_TRAIN else 10 # 50   

def get_model():
    inputs_1 = tf.keras.Input(shape=(82,))  
        
    features_1 = VariableSelectionFlow(82, units_1, drop_1, dense_units=dense_units)(inputs_1)
    features_2 = VariableSelectionFlow(units_1, units_2, drop_2)(features_1)         
    features_3 = VariableSelectionFlow(units_2, units_3, drop_3)(features_2)         

    outputs = L.Dense(4, activation="softmax")(features_3)
    
    model = Model(inputs=inputs_1, outputs=outputs)
    
    return model

In [ ]:
def weighted_categorical_crossentropy(class_weight):
    def loss(y_true, y_pred):
        y_true = tf.dtypes.cast(y_true, tf.float64) 
        sample_weight = tf.math.multiply(class_weight, y_true)
        sample_weight = tf.reduce_sum(sample_weight, axis=-1)     
        cce = tf.keras.losses.CategoricalCrossentropy()
        losses = cce(y_true, y_pred, sample_weight=sample_weight)
        return losses
    
    return loss    

In [ ]:
def ordinal_weighted_categorical_crossentropy(class_weight):
    def loss(y_true, y_pred):
        ord_weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1)) / (K.int_shape(y_pred)[1] - 1), dtype='float32')
        
        y_true = tf.dtypes.cast(y_true, tf.float64) 
        sample_weight = tf.math.multiply(class_weight, y_true)
        sample_weight = tf.reduce_sum(sample_weight, axis=-1)     
        cce = tf.keras.losses.CategoricalCrossentropy()
        losses = cce(y_true, y_pred, sample_weight=sample_weight)  
        alpha = 0.8
        return (1.0 + ord_weights * alpha) * losses
    
    return loss    

In [ ]:
if USE_MY_TRAIN:
    train_df = pd.read_csv('/kaggle/input/piu-train/my_train.csv')    
    train_df['sii_with_difficulty'] = train_df['sii']
    train_df.loc[train_df['score_real_sii'] < 0.15, 'sii_with_difficulty'] = train_df['sii'] + 10

In [ ]:
train_df['sii'] = train_df['sii'].astype(int)
X_train_size = train_df.shape[0]

if VALID:
    oof_preds = np.zeros((X_train_size, 4))
    oof_preds_sii = np.zeros(X_train_size)
    folds = np.zeros(X_train_size)

if TEST:
    preds_test = np.zeros((n_splits, test_df.shape[0], 4))
    preds_test_sii = np.zeros(test_df.shape[0])   

In [ ]:
if VALID:
    my_train_df = pd.read_csv('/kaggle/input/piu-train/my_train.csv')
    train_df['sii_with_difficulty'] = my_train_df['sii']
    train_df['score_real_sii'] = my_train_df['score_real_sii']
    train_df.loc[train_df['score_real_sii'] < 0.15, 'sii_with_difficulty'] = train_df['sii'] + 10

In [ ]:
if FULL_TRAIN:
    n_models = 15
    preds_test_models = np.zeros((n_models, test_df.shape[0], 4))

    if VALID:
        X_train = train_df.drop(['id', 'sii', 'score_real_sii', 'sii_with_difficulty'], axis=1)
    else:
        X_train = train_df.drop(['id', 'sii'], axis=1)
    y_train = pd.get_dummies(train_df['sii'], dtype=float)
    
    class_weights = np.array([0.06, 0.12, 0.18, 0.64])
    wcc = weighted_categorical_crossentropy(class_weights)   
    # wcc = ordinal_weighted_categorical_crossentropy(class_weights)
      
    for i in range(n_models):  
        if n_models > 1:
            set_randoms(i)

        model = get_model()
        model.compile(
                optimizer=Adam(learning_rate=learning_rate, epsilon=1e-7),  
                loss=wcc
            ) 

        print(f"Start training the model {i} ...")
        model.fit(
                x=X_train,
                y=y_train,
                batch_size=batch_size,
                epochs=num_epochs,
                shuffle=True
            )
        print(f"Model training finished {i}.")

        if TEST:
            preds_test_models[i] = model.predict(test_df.values) 
            
    if TEST:
        preds_test_full_train = np.mean(preds_test_models, axis=0)
        preds_test_sii_full_train = np.argmax(preds_test_full_train, axis=1)

    preds_train = model.predict(X_train)
    y_train_sii = np.argmax(y_train, axis=1)
    preds_sii_train = np.argmax(preds_train, axis=1)
    res_train = cohen_kappa_score(y_train_sii, preds_sii_train, weights="quadratic")
    print('TRAIN_RES = ', res_train, '\n')

In [ ]:
if VALID:
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    for fold,(train_idx, valid_idx) in enumerate(SKF.split(train_df, train_df['sii_with_difficulty'])):
        
        set_randoms(27)  

        print('#'*25)
        print('### Fold',fold + 1)
        print('### Train size', train_idx.shape[0], 'Valid size', valid_idx.shape[0])
        print('#'*25)

        X_train = train_df.drop(['id', 'sii', 'score_real_sii', 'sii_with_difficulty'], axis=1).loc[train_idx]
        y_train = pd.get_dummies(train_df.loc[train_idx,'sii'], dtype=float)
        X_valid = train_df.drop(['id', 'sii', 'score_real_sii', 'sii_with_difficulty'], axis=1).loc[valid_idx]
        y_valid = pd.get_dummies(train_df.loc[valid_idx, 'sii'], dtype=float)

        class_weights = np.array([0.06, 0.12, 0.18, 0.64])
        # wcc = weighted_categorical_crossentropy(class_weights)  # Sacar
        wcc = ordinal_weighted_categorical_crossentropy(class_weights)

        model = get_model()

        model.compile(
            optimizer=Adam(learning_rate=learning_rate, epsilon=1e-7),  
            loss=wcc
        )

        # Create an early stopping callback
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True     
         )
        lr = ReduceLROnPlateau(monitor="val_loss", mode='min', factor=0.95, patience=1, verbose=0) 

        print("Start training the model...")
        history = model.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            epochs=num_epochs,
            shuffle=True,
            validation_data=(X_valid, y_valid),
            callbacks=[lr]
            # callbacks=[lr, early_stopping]   
        )
        print("Model training finished.")
        val_loss_hist = history.history['val_loss']
        print(f'best epoch: {np.argmin(val_loss_hist)}')

        if VALID:
            preds = model.predict(X_valid)

            y_valid_sii = np.argmax(y_valid, axis=1)
            preds_sii = np.argmax(preds, axis=1)

            preds_train = model.predict(X_train)
            y_train_sii = np.argmax(y_train, axis=1)
            preds_sii_train = np.argmax(preds_train, axis=1)
            res_train = cohen_kappa_score(y_train_sii, preds_sii_train, weights="quadratic")
            print('TRAIN_RES = ', res_train, '\n')
            
            res = cohen_kappa_score(y_valid_sii, preds_sii, weights="quadratic")
            print('RES = ', res, '\n')

            oof_preds[valid_idx] = preds
            oof_preds_sii[valid_idx] = preds_sii
            folds[valid_idx] = fold

        if TEST:
            preds_test[fold] = model.predict(test_df.values)  
            
        model.save_weights(f'mod_v64_f{fold}_res{res:.4f}.weights.h5')
        
    if TEST:
        preds_test_cv = np.mean(preds_test, axis=0)
        preds_test_sii_cv = np.argmax(preds_test_cv, axis=1)

    print('#'*25)
    res = cohen_kappa_score(train_df.sii, oof_preds_sii, weights="quadratic")
    print('OVERALL RES =', res,'\n')

In [ ]:
if SAVE_MY_TRAIN:
    train_df['score_real_sii'] = None
    for i in range(train_df.shape[0]):
        train_df.loc[i, 'score_real_sii'] = oof_preds[i, train_df.loc[i, 'sii']]
    train_df.to_csv('my_train.csv', index=False)

In [ ]:
if VALID:
    tot = np.zeros(4)
    tot_real = np.zeros(4)
    for i in range(oof_preds_sii.shape[0]):
        tot[int(oof_preds_sii[i])] += 1
        tot_real[int(train_df.loc[i, 'sii'])] += 1
    print(f'TOT REAL: {tot_real}')
    print(f'TOT PRED: {tot}')

    oof_preds_sii = oof_preds_sii.astype(int)

    c = confusion_matrix(train_df['sii'].values, oof_preds_sii)
    cm_display = ConfusionMatrixDisplay(confusion_matrix=c, display_labels=[0, 1, 2, 3])
    cm_display.plot()
    plt.show()

In [ ]:
if TEST:
    if FULL_TRAIN and VALID:
        preds_test = 0.5 * preds_test_cv + 0.5 * preds_test_full_train
    elif FULL_TRAIN:
        preds_test = preds_test_full_train
    else:
        preds_test = preds_test_cv
    preds_test_sii = np.argmax(preds_test, axis=1)

In [ ]:
if TEST and FULL_TRAIN and VALID:
    res = cohen_kappa_score(preds_test_sii_full_train, preds_test_sii_cv, weights="quadratic")
    print('KAPPA FULL_TRAIN - CV = ', res,'\n')
    
    res = cohen_kappa_score(preds_test_sii_full_train, preds_test_sii, weights="quadratic")
    print('KAPPA FULL_TRAIN - PRED = ', res,'\n')

    res = cohen_kappa_score(preds_test_sii_cv, preds_test_sii, weights="quadratic")
    print('KAPPA CV - PRED = ', res,'\n')

In [ ]:
if TEST:
    test_df['sii'] = preds_test_sii
    test_df['sii'].to_csv('submission.csv')

In [ ]:
test_df.head(20)

In [ ]:
!zip files.zip *

os.chdir(r'/kaggle/working')
from IPython.display import FileLink

FileLink(r'files.zip')